# **Notebook D.** Classification Using a Transformer Model
----

One of the biggest recent developments in Natural Language Processing has come from the introduction of Transformer models (e.g. *BERT, RoBERTa*, etc.). The idea is that a model is trained on a very large corpus and used to create an embedding represention of words. This "raw" model can be downloaded and then fine-tuned (retrained) on our own data.

There are several ways to implement these models. Researchers who are most comfortable with Python may start with the **Transformers** library by **HuggingFace** (https://huggingface.co/transformers/). This is the most flexible approach, but it also requires effort for researchers to implement.

An alterantive is the **SimpleTransformers** library which is a wrapper for this functionality. This library contains an easy-to-use version of this transformer technique (https://simpletransformers.ai) that is similar to the sklearn commands we have used thus far.

This package is not pre-installed with colab. To do this, we need perform the following:
 - 1) run *!pip install simpletransformers* in the notebook below
 - 2) Comment out the code by putting a # in front of the line (e.g. *#!pip install simpletransformers*)
 - 3) Rerun all of the code from the top menu (or hit Ctrl+F9)




Models: https://simpletransformers.ai/docs/classification-specifics/#supported-model-types

Model Options: https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model


In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 9.7 MB/s eta 0:00:00
   ━━━

In [ ]:
# Load the Simple Transformers Package for Text Classification

from simpletransformers.classification import ClassificationModel

In [ ]:
# Turn of warnings, just to avoid pesky messages that might cause confusion here
# Remove when testing your own code #
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging

logging.basicConfig(level=logging.ERROR)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

# D.1. Preamble: Load Packages
---

In [ ]:
# General Packages #
import os
import pandas as pd
import numpy as np

# TQDM to Show Progress Bars #
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

# SKLearn libraries for splitting sample and validation
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Additional Libraries that we are using only in this notebook
import torch
import gc

In [ ]:
# Turn of warnings, just to avoid pesky messages that might cause confusion here
# Remove when testing your own code #
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Mount Personal Google Drive on own Machine -- You have to follow the link to log in #
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# D.2. Load Training Data ##
----------------

We are going to use the data on the Google drive. This is in a csv file, and so we are going to load the data as a dataframe, and then convert the main data (Text Ids, Indicator for Tech / Non-Tech, Text Abstract) from a Pandas DataFrame to a list (which is more easily used in later sections).

In [ ]:
# Change to Working Directory with Training Data #
os.chdir("/content/drive/MyDrive/Power-data-main/")

# Load Training Data #
file_path = 'Training Data/power_data.tsv'

# Read the TSV file
df = pd.read_csv(file_path, sep='\t')

# Extract the required columns: id, cleaned_abstract, and actual
IDs = np.array(df['id'].values.tolist())
Abstract_Text = df['abstract'].values.tolist()
Classes = df['actual'].values.tolist()

# D.3. Perform Classification with Transformer Model
---

As before, we are going to go through different models and compare their performance. Recall that transformer models are pre-trained by an external entity and we are simply downloading them (pre-trained) from the web and fine tuning them our particular application.

We download these models from hugging face. We are using the simpletransformers library which allows us to automatically download and train these models, using the same basic command for different models. We simply need to specify the model architecture (e.g. BERT) and then specific model, which usually refers to the type of data it was trained on (e.g. bert-base-uncased).

In the following link you can see the possible models that can be used by simple-transformers.

* https://simpletransformers.ai/docs/classification-specifics/#supported-model-types

This refers to the model type or architecture. There might be various types of models trained for different purposes that use the same architecture. You can downlaod the most common models directly from huggingface:

* https://huggingface.co/transformers/pretrained_models.html

You can also download community models here:

* https://huggingface.co/models

Below we define a list of the different transformer models we are going to use. These are listed in the following order: Name (e.g. BERT (Chinese)), Architecture (e.g. bert), Specific Model (e.g. bert-base-chinese)


In [ ]:
CLASSIFIERS = [
 ["BERT (Chinese)", "bert", "bert-base-chinese"],
 ["RoBERTa (Chinese)", "roberta", "hfl/chinese-roberta-wwm-ext"],
 ["XLNet (Chinese)", "xlnet", "hfl/chinese-xlnet-base"],
 ["ELECTRA (Chinese)", "electra", "hfl/chinese-electra-base-discriminator"],
 ["MacBERT (Chinese)", "bert", "hfl/chinese-macbert-base"],
 ["BERT-wwm (Chinese)", "bert", "hfl/chinese-bert-wwm"],
]



In [ ]:
NUM_OF_SPLITS = 5

# Define whether you want to manually reweight the sample by oversampling the smaller class
Reweight = True

# Define arrays in which to store classification outputs #
RESULTS = []
Classified_Values =[]

# Loop Through Different Classifiers #
for CL in tqdm_notebook(CLASSIFIERS, desc = "Evaluating Classifiers", leave = True):

    # Extract Classifier Names & Model #
    name  = CL[0]
    Model1 = CL[1]
    Model2 = CL[2]

    # Define Arrays to store Actual, Predicted and Ids variables (Because we are shuffling them in next step) #
    y_actual = []
    y_predicted = []
    id_s = []

    # Loop through K Folds and Repeat Cross Validation #

    KFoldSplitter = StratifiedKFold(n_splits = NUM_OF_SPLITS, shuffle = True, random_state = 1)

    for train_i, test_i in tqdm_notebook(KFoldSplitter.split(Abstract_Text, Classes),
                                            desc = 'Cross-Validating',
                                            leave = False,
                                            total = NUM_OF_SPLITS):

        # Select Rows in Data Based on Indexes [train_i, test_i]
        Y = np.array(Classes)

        Abstract_Text_Array = np.array(Abstract_Text)

        train_X, test_X = Abstract_Text_Array[train_i], Abstract_Text_Array[test_i]
        train_y, test_y = Y[train_i], Y[test_i]
        Train_IDs, Test_IDs = IDs[train_i], IDs[test_i]

        # Create Training Data in Paired Format (Nessesary for Transformers) #
        TrainingDataframe = list(zip( list(train_X), list(train_y)))
        TestDataframe = list(zip( list(test_X), list(test_y)))

        train_df = pd.DataFrame(TrainingDataframe)
        train_df.columns = ["text", "labels"]

        # Create a Classification Model
        model = ClassificationModel(Model1, Model2,
                                    num_labels=2,
                                    args={'num_train_epochs':5,
                                          'overwrite_output_dir': True,
                                          'use_early_stopping':False,
                                          'use_cuda':True,
                                          'train_batch_size':50,
                                          'do_lower_case':True,
                                          'silent':True,
                                          'no_cache':True,
                                          'no_save':True}
                                    )

    # Train the Model
    model.train_model(train_df)

    # Predict on Holdout Sample #
    predictions, raw_outputs = model.predict( list(test_X) )

    # Store Output #
    id_s = id_s + list(Test_IDs)
    y_actual = y_actual + list(test_y)
    y_predicted = y_predicted + list(predictions)

    gc.collect()
    torch.cuda.empty_cache()

    # ---------------------------------------------------------- #
    # This runs only after all of the folds have been classified #
    # ---------------------------------------------------------- #

    # Compute the Share of Tech Texts #
    Share = np.round(np.mean(y_predicted), 3)

    # Calculate Model Performance Metrics #
    Accuracy = accuracy_score(y_actual, y_predicted)
    ROC = roc_auc_score(y_actual, y_predicted)
    Precision = precision_score(y_actual, y_predicted)
    Recall = recall_score(y_actual, y_predicted)
    F1 = f1_score(y_actual, y_predicted)
    CM = confusion_matrix(y_actual, y_predicted)

    # Round to 3 Decimal Places #
    #FN = np.round(CM[0][0]/CM[0].sum(), 3)
    #FP = np.round(CM[0][1]/CM[0].sum(), 3)
    #TN = np.round(CM[1][0]/CM[1].sum(), 3)
    #TP = np.round(CM[1][1]/CM[1].sum(), 3)

    FN = np.round(CM[0][0]/(CM[0][0] + CM[1][0]), 3)
    FP = np.round(CM[0][1]/(CM[0][1] + CM[1][1]), 3)
    TN = np.round(CM[1][0]/(CM[0][0] + CM[1][0]), 3)
    TP = np.round(CM[1][1]/(CM[0][1] + CM[1][1]), 3)

    # Add Classification Performance Metrics to List#
    RESULTS.append([name, Share, TP, FN, FP, TN,
                                            np.round(Accuracy, 3),
                                            np.round(ROC, 3),
                                            np.round(Precision, 3),
                                            np.round(Recall, 3),
                                            np.round(F1, 3)])


    # Add Classification Results to List #
    Classified_Values.append(list(zip(len(id_s)*[name],id_s, y_actual, y_predicted)))

Evaluating Classifiers:   0%|          | 0/6 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

Cross-Validating:   0%|          | 0/5 [00:00<?, ?it/s]

# D.4. Output Classification Results #
----


In [ ]:
# Convert List to Dataframe #
RESULTS_TABLE = pd.DataFrame(RESULTS, columns = ["Name", "Share", "True-Positives",
                                                 "False-Negatives", "False-Positives",
                                                 "True-Negatives","Accuracy", "AUC",
                                                 "Precision", "Recall", "F1"] )

RESULTS_TABLE["Type"] = "Transformer"
RESULTS_TABLE = RESULTS_TABLE[["Name", "Type", "Share", "True-Positives",
                               "False-Negatives", "False-Positives",
                               "True-Negatives","Accuracy", "AUC",
                               "Precision", "Recall", "F1"]]



# Output Results #
RESULTS_TABLE.sort_values("Accuracy", ascending = False ).to_csv("./Output/Model Performance/Transformer Classification Model Performance.csv")

# Display Results -- Out of Sample (Holdout) prediction -- Sorted by Accuracy #
RESULTS_TABLE.sort_values("Accuracy", ascending = False )


,Name,Type,Share,True-Positives,False-Negatives,False-Positives,True-Negatives,Accuracy,AUC,Precision,Recall,F1
2,XLNet (Chinese),Transformer,0.423,0.927,0.936,0.073,0.064,0.932,0.930,0.927,0.915,0.921
5,BERT-wwm (Chinese),Transformer,0.438,0.909,0.944,0.091,0.056,0.928,0.928,0.909,0.926,0.917
3,ELECTRA (Chinese),Transformer,0.432,0.911,0.938,0.089,0.062,0.927,0.926,0.911,0.918,0.915
0,BERT (Chinese),Transformer,0.441,0.904,0.945,0.096,0.055,0.927,0.927,0.904,0.928,0.916
4,MacBERT (Chinese),Transformer,0.445,0.891,0.941,0.109,0.059,0.919,0.920,0.891,0.924,0.908
1,RoBERTa (Chinese),Transformer,0.401,0.900,0.886,0.100,0.114,0.892,0.885,0.900,0.841,0.869


In [ ]:
# Output Classification Results for Training Dataset -- PREDICTED VALUES -- Out Of Sample (Holdout) Prediction #

for i in range(0,len(Classified_Values), 1):

  Temp = pd.DataFrame(  Classified_Values[i],
                        columns = ['Model', 'id', 'Actual', 'Predicted'] )

  if i == 0:
    name = Temp.head(1)['Model'][0]
    Temp = Temp[['id', 'Actual', 'Predicted']]
    Temp.columns = ['id', 'Actual', name]
    Final = Temp

  else:

    name = Temp.head(1)['Model'][0]
    Temp = Temp[['id', 'Predicted']]
    Temp.columns = ['id', name]

    Final = Final.merge(Temp, on = ['id'])

# Save Data Frame #
Final.to_csv("./Output/Classification Output/Transformer Classification Results.csv")

Delete files that were created behind the scenes by the transformer model.

In [ ]:
rm -rf "./outputs"

In [ ]:
rm -rf "./runs"